In [1]:
!nvidia-smi

Mon Apr  7 20:24:04 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA RTX A4000               On  |   00000000:01:00.0 Off |                  Off |
| 41%   39C    P8             15W /  140W |       2MiB /  16376MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
import torch
import os

# Set memory allocation to be more efficient
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

In [8]:
!df -h

Filesystem                            Size  Used Avail Use% Mounted on
overlay                                16G  7.0G  9.1G  44% /
tmpfs                                  64M     0   64M   0% /dev
shm                                    15G     0   15G   0% /dev/shm
/dev/nvme0n1p1                        3.7T  913G  2.8T  25% /etc/hosts
/dev/mapper/ubuntu--vg--1-ubuntu--lv  1.8T   35G  1.7T   3% /usr/bin/nvidia-smi
tmpfs                                  63G     0   63G   0% /sys/fs/cgroup
tmpfs                                  63G   12K   63G   1% /proc/driver/nvidia
tmpfs                                  63G  4.0K   63G   1% /etc/nvidia/nvidia-application-profiles-rc.d
tmpfs                                  13G   14M   13G   1% /run/nvidia-persistenced/socket
tmpfs                                  63G     0   63G   0% /proc/asound
tmpfs                                  63G     0   63G   0% /proc/acpi
tmpfs                                  63G     0   63G   0% /proc/scsi
tmpfs          

In [4]:
!du -hs /* 2>/dev/null | sort -rh | head -10

11G	/usr
7.5G	/venv
5.2G	/root
1.8G	/opt
112M	/var
28M	/train.jsonl
3.9M	/wandb
2.8M	/validation.jsonl
2.3M	/etc
1.1M	/test_ood_1e5.jsonl


In [12]:
!du -sh * | sort -h

du: cannot read directory 'proc/84/task/84/fdinfo': Permission denied
du: cannot read directory 'proc/84/map_files': Permission denied
du: cannot read directory 'proc/84/fdinfo': Permission denied
du: cannot read directory 'proc/115/task/115/fdinfo': Permission denied
du: cannot read directory 'proc/115/map_files': Permission denied
du: cannot read directory 'proc/115/fdinfo': Permission denied
du: cannot read directory 'proc/116/task/116/fdinfo': Permission denied
du: cannot read directory 'proc/116/map_files': Permission denied
du: cannot read directory 'proc/116/fdinfo': Permission denied
du: cannot read directory 'proc/134/task/134/fdinfo': Permission denied
du: cannot read directory 'proc/134/task/136/fdinfo': Permission denied
du: cannot read directory 'proc/134/task/137/fdinfo': Permission denied
du: cannot read directory 'proc/134/task/138/fdinfo': Permission denied
du: cannot read directory 'proc/134/task/139/fdinfo': Permission denied
du: cannot read directory 'proc/134/task/

In [23]:
!du -sh /usr/* | sort -h

0	/usr/games
0	/usr/lib64
0	/usr/libx32
4.0K	/usr/etc
92K	/usr/src
1.6M	/usr/libexec
9.4M	/usr/sbin
15M	/usr/lib32
27M	/usr/include
174M	/usr/bin
326M	/usr/share
4.6G	/usr/local
5.9G	/usr/lib


In [5]:
import shutil
import os

# Clear Hugging Face cache
!rm -rf ~/.cache/huggingface/hub

# Or selectively remove specific model caches
!rm -rf ~/.cache/huggingface/hub/models--openai-community--gpt2-large

In [6]:
!pip cache purge

Files removed: 13


In [7]:
!sync && echo 3 | sudo tee /proc/sys/vm/drop_caches

tee: /proc/sys/vm/drop_caches: Read-only file system
3


In [24]:
# Uninstall potentially conflicting packages for a clean slate
!pip uninstall -y transformers torch torchvision torchaudio tensorflow tf-keras datasets accelerate

# Install PyTorch with CUDA support (adjust cuXXX if needed for your environment)
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

# Install the LATEST stable versions of transformers and other required libraries
!pip install transformers datasets wandb huggingface_hub accelerate tensorflow tf-keras

# Verify installation
!pip list | grep -E 'torch|transformers|datasets|accelerate'
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA version used by torch: {torch.version.cuda}")
import transformers
print(f"Transformers version: {transformers.__version__}")
import accelerate
print(f"Accelerate version: {accelerate.__version__}")

Found existing installation: transformers 4.51.0
Uninstalling transformers-4.51.0:
  Successfully uninstalled transformers-4.51.0
Found existing installation: torch 2.5.1+cu121
Uninstalling torch-2.5.1+cu121:
  Successfully uninstalled torch-2.5.1+cu121
Found existing installation: torchvision 0.20.1+cu121
Uninstalling torchvision-0.20.1+cu121:
  Successfully uninstalled torchvision-0.20.1+cu121
Found existing installation: torchaudio 2.5.1+cu121
Uninstalling torchaudio-2.5.1+cu121:
  Successfully uninstalled torchaudio-2.5.1+cu121
Found existing installation: tensorflow 2.19.0
Uninstalling tensorflow-2.19.0:
  Successfully uninstalled tensorflow-2.19.0
Found existing installation: tf_keras 2.19.0
Uninstalling tf_keras-2.19.0:
  Successfully uninstalled tf_keras-2.19.0
Found existing installation: datasets 3.5.0
Uninstalling datasets-3.5.0:
  Successfully uninstalled datasets-3.5.0
Found existing installation: accelerate 1.6.0
Uninstalling accelerate-1.6.0:
  Successfully uninstalled a

In [26]:
from huggingface_hub import notebook_login
notebook_login()

In [27]:
# Import necessary libraries
import os
import wandb
from datasets import load_dataset
from transformers import (
    GPT2Tokenizer,
    GPT2LMHeadModel,
    Trainer,
    TrainingArguments,
    DataCollatorForLanguageModeling
)
import torch

torch.manual_seed(9001)
import random
random.seed(9001)

# Initialize wandb (make sure you are logged in with `wandb login`)
wandb.init(project="gpt2-large-countdown", name="gpt2-large-countdown-run-1", reinit=True)

2025-04-07 20:30:46.226623: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744057846.245457     432 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744057846.251174     432 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1744057846.265966     432 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1744057846.265984     432 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1744057846.265986     432 computation_placer.cc:177] computation placer alr

In [28]:
# Load the datasets
data_files = {
    "train": "train.jsonl",
    "validation": "validation.jsonl",
    "test_1e3": "test_ood_1e3.jsonl"
}
dataset = load_dataset("json", data_files=data_files)

# Combine the fields into a single prompt.
def preprocess(example):
    """ Here we combine numbers, target, and the chain-of-thought (CoT) into one string. """
    cot_text = " ".join(example["cot"])
    text = f"""A conversation between User and Assistant. The user asks a question, and the Assistant solves it. The assistant first thinks about the reasoning process in the mind and then provides the user with the answer.
User: Using the numbers {example['numbers']}, create an equation that equals {example['target']}. You can use basic arithmetic operations (+, -, *, /) and each number can only be used once. Show your work in <think> </think> tags. And return the final answer in <answer> </answer> tags, for example <answer> (1 + 2) / 3 </answer>.
Assistant: Let me solve this step by step.
<think>{cot_text}</think>
<answer>{example['target']}</answer>"""
    return {"text": text}

# Map the preprocessing function over the dataset splits
dataset = dataset.map(preprocess, batched=False)

In [29]:
# Load the GPT-2 small tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2-large")
# GPT-2 doesn't have a pad token by default; we set it equal to the EOS token.
tokenizer.pad_token = tokenizer.eos_token

# Tokenize the dataset
def tokenize_function(example):
    return tokenizer(example["text"], truncation=True, padding="max_length", max_length=256)

tokenized_datasets = dataset.map(tokenize_function, batched=True, remove_columns=["numbers", "target", "cot", "text"])

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/666 [00:00<?, ?B/s]

In [30]:
# Data collator for causal language modeling (no MLM masking needed)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [31]:
# Define training arguments
training_args = TrainingArguments(
    output_dir="./gpt2-large-countdown",
    eval_strategy="steps", # Evaluate at set steps instead of at the end of each epoch
    eval_steps=500, # Evaluate every 500 steps
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=2,
    num_train_epochs=5,
    learning_rate=3e-5,
    weight_decay=0.01,
    logging_steps=100,
    save_steps=1000,
    save_total_limit=1,
    save_only_model=True,
    fp16=True,
    gradient_checkpointing=True,
    push_to_hub=True, # Automatically push checkpoints and final model to the Hub
    hub_model_id="giordanorogers/gpt2-large-countdown", # Replace with your repo ID
    load_best_model_at_end=True, # Automatically load the best model based on eval_loss
    metric_for_best_model="eval_loss",
    report_to=["wandb"]
)

# Skip the TensorFlow conversion entirely
model = GPT2LMHeadModel.from_pretrained(
    "openai-community/gpt2-large",
    use_safetensors=True,
    device_map="auto"  # Distribute across available devices
)
# Set the pad_token_id to avoid warnings during training
model.config.pad_token_id = tokenizer.pad_token_id

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
)

config.json:   0%|          | 0.00/666 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/3.25G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [32]:
trainer.train()

trainer.push_to_hub()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss,Validation Loss
500,0.241000,0.229880


RuntimeError: [enforce fail at inline_container.cc:603] . unexpected pos 533867328 vs 533867220

In [ ]:
import torch
import gc

# Print tensors in memory
for obj in gc.get_objects():
    try:
        if torch.is_tensor(obj) and obj.is_cuda:
            print(f"{type(obj).__name__} - Size: {obj.size()} - {obj.dtype} - {obj.device} - {obj.element_size() * obj.nelement() / 1024 / 1024:.2f} MB")
    except:
        pass

In [18]:
# Clear GPU cache
torch.cuda.empty_cache()

# Force garbage collection
import gc
gc.collect()

37260

In [ ]:
# Evaluate the model on the OOD test set
eval_results = trainer.evaluate(eval_dataset=tokenized_datasets["test"])
print("OOD Test Evaluation:", eval_results)

In [ ]:
from transformers import pipeline, GPT2Tokenizer, GPT2LMHeadModel

# Load the trained model and tokenizer from the local directory (or replace with your Hub repo ID)
model = GPT2LMHeadModel.from_pretrained("giordanorogers/gpt2-large-countdown")  # or "giordanorogers/gpt2-small-countdown" if pushed
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token  # Ensure pad_token is set

# Create a text generation pipeline
generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

# Define a prompt to test your model
numbers = [16, 44, 57, 62, 71, 75]

target = 1557

prompt = f"""A conversation between User and Assistant. The user asks a question, and the Assistant solves it. The assistant first thinks about the reasoning process in the mind and then provides the user with the answer.
User: Using the numbers {numbers}, create an equation that equals {target}. You can use basic arithmetic operations (+, -, *, /) and each number can only be used once. Show your work in <think> </think> tags. And return the final answer in <answer> </answer> tags, for example <answer> (1 + 2) / 3 </answer>.
Assistant: Let me solve this step by step.
<think>"""

# Generate output
output = generator(prompt, max_length=300, truncation=True, num_return_sequences=1)

# Print the generated outputa
print(output[0]["generated_text"])